# Force analysis of ActuAtor

Summary of protocol for estimating forces on the membrane. Nuclei were traced from the EM images manually using Fiji. Assuming the Helfrich-Canham-Evans and surface energy of the membrane,

$$E =  \underbrace{\int_\Omega \kappa_b H^2 dA}_{E_b} + \underbrace{\int_\Omega \sigma dA}_{E_\sigma},$$

where $\kappa_b$ is the bending modulus, $H$ is the mean curvature given by the two principal curvatures, $(k_1 + k_2)/2$,  $\sigma$ is the prescribed constant surface tension, and $dA$ the area element. Given that the segmented EM images are plane curves, we assume naively that the out of plane principal curvature is naught ($k_2 = 0$). Rewriting as an energy density (per length) and simplifying,

$$\frac{E_b}{l} =\frac{\kappa_b}{4} \int_C k_1^2 ds,$$

and 

$$\frac{E_\sigma}{l} =  \sigma \int_C ds,$$ 


where $l$ is the depth relating the integrals over area, $\Omega$, and the plane curve, $C$; $ds$ is the arc-length element. To obtain the areal force density $f$ such that $f ds = -\nabla (E/l)$, we use automatic differentiation using `jax`. Note that $f$ is independent of the length scale $l$.  

## Analysis

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import math
from collections import defaultdict
from functools import partial
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from jax import jit

from jax.config import config
config.update("jax_enable_x64", True)

from PIL import Image
from scipy.interpolate import splev, splprep
from tqdm.notebook import tqdm

import automembrane.util as u
from automembrane.energy import ClosedPlaneCurveMaterial
from automembrane.integrator import fwd_euler_integrator

u.matplotlibStyle(small=10, medium=12, large=14)

# List of segmentations as Paths
files = list(
    map(
        Path,
        [
            f"coordinates/{i}"
            for i in [
                "cell1/34D-grid2-s3-acta1_001_16.txt",
                "cell2/34D-grid3-ActA1_007_16.txt",
                "cell2/34D-grid3-ActA1_013_16.txt",
                "cell3/34D-grid2-s2_002_16.txt",
                "cell3/34D-grid2-s5_005_16.txt",
                "cell3/34D-grid3-ActA1_020_16.txt",
                "cell3/34D-grid3-s6_005_16.txt",
                "cell4/34D-grid2-s3_028_16.txt",
                "cell5/34D-grid3-ActA1_001_16.txt",
                "cell5/34D-grid3-ActA1_002_16.txt",
                "cell5/34D-grid3-ActA1_003_16.txt",
                "cell5/34D-grid3-ActA1_004_16.txt",
            ]
        ],
    )
)

# Map of image key to microns per pixel
images = {
    "34D-grid2-s3-acta1_001_16": 0.012723,
    "34D-grid3-ActA1_007_16": 0.015904,
    "34D-grid3-ActA1_013_16": 0.015904,
    "34D-grid2-s2_002_16": 0.015904,
    "34D-grid2-s5_005_16": 0.015904,
    "34D-grid3-ActA1_020_16": 0.015904,
    "34D-grid3-s6_005_16": 0.015904,
    "34D-grid2-s3_028_16": 0.015904,
    "34D-grid3-ActA1_001_16": 0.015904,
    "34D-grid3-ActA1_002_16": 0.015904,
    "34D-grid3-ActA1_003_16": 0.015904,
    "34D-grid3-ActA1_004_16": 0.015904,
}


Assuming bending modulus $1\times10^{-19}$ J and modest membrane tension $0.05$ mN/m.

In [ ]:
parameters = {
    "Kb": 0.1 / 4,  # Bending modulus (pN um; original 1e-19 J)
    "Ksg": 50,  # Global stretching modulus (pN um/um^2; original 0.05 mN/m)
    "Ksl": 1,
}

mem = ClosedPlaneCurveMaterial(**parameters)

In [ ]:
## Benchmarking playground
coords = np.loadtxt(files[0])
coords = np.vstack((coords, coords[0]))

# e = mem.energy(coords)
# f = mem.force(coords)

# e, f = mem.energy_force(coords)

# print("Energy:", e, type(e), e.shape, sum(e))
# print("")
# print("Forces by jacrev:", type(f), f.shape, sum(f).shape)

# a, vjp = jax.vjp(mem._energy, coords)
# (J,) = jax.vmap(vjp, in_axes=0)(jax.numpy.eye(len(a)))

# J_fromvjp, = vjp(np.array([1.,1.,1.]))

# print("")
# print("allclose(J, f):", np.allclose(J, f))

# print("")
# print("J_vjp, f:", np.allclose(J_fromvjp, sum(f)), np.amax(np.abs(J_fromvjp-sum(f))))

# j0, = vjp(np.array([1.,0.,0.]))
# j1, = vjp(np.array([0.,1.,0.]))
# j2, = vjp(np.array([0.,0.,1.]))

# print(j0.shape, j1.shape, j2.shape)

# print("")
# print(np.allclose(sum(f), (j0+j1+j2)))
# print(np.allclose(j0+j1+j2, J_fromvjp))
# print(np.allclose(sum(f), J_fromvjp))


We iterate over the segmentations. To ensure that the discrete curves are sufficiently sampled, we parameterize the curve using a B-spline and resample evenly. We perform `n_iter` steps of forward Euler energy relaxation to smooth out artifacts of the discretization. We choose the sum of half the length of each incident edge to be the dual length, $ds_i$, of the $i^{\text{th}}$ vertex, $v_i$.

In [ ]:
# n_vertices = 800  # Number of vertices in resampling
target_edge_length = 0.05  # target edge length in um for resampling

total_time = 0.005
dt = 5e-6  # Timestep
n_iter = math.floor(total_time / dt)  # Number of relaxation steps
data = defaultdict(dict)

for file in files:
    k = file.stem

    print("Processing:", k)
    original_coords = np.loadtxt(file)
    original_coords = np.vstack(
        (original_coords, original_coords[0])
    )  # Energy expects last point to equal first
    data[k]["original_coords"] = original_coords

    total_length = np.sum(
        np.linalg.norm(
            np.roll(original_coords[:-1], -1, axis=0) - original_coords[:-1], axis=1
        )
    )

    n_vertices = math.floor(total_length / target_edge_length)
    print(f"  Resampling to {n_vertices} vertices")
    # Periodic cubic B-spline interpolation with no smoothing (s=0)
    tck, _ = splprep([original_coords[:, 0], original_coords[:, 1]], s=0, per=True)
    data[k]["spline"] = tck

    xi, yi = splev(np.linspace(0, 1, n_vertices), tck)
    coords = np.hstack((xi.reshape(-1, 1), yi.reshape(-1, 1)))

    # Perform energy relaxation
    relaxed_coords = coords
    if n_iter > 0:
        relaxed_coords, energy_log = fwd_euler_integrator(
            relaxed_coords, mem, n_iter, dt
        )
        print(
            f"  DELTA E: {energy_log[-1] - energy_log[0]}; E_before: {energy_log[0]}; E_after: {energy_log[-1]}"
        )

    data[k]["relaxed_coords"] = relaxed_coords

    # Compute dual length
    dc = np.roll(relaxed_coords[:-1], -1, axis=0) - relaxed_coords[:-1]
    edgeLengths = np.linalg.norm(dc, axis=1)
    dualLengths = ((edgeLengths + np.roll(edgeLengths, 1)) / 2.0).reshape(-1, 1)
    dualLengths = np.vstack((dualLengths, dualLengths[0]))
    data[k]["dualLengths"] = dualLengths

    data[k]["relaxed_forces"] = -sum(mem.force(relaxed_coords)) / dualLengths


In [ ]:
cm = mpl.cm.viridis_r
padding = 2  # padding to add around cell boundary to give broader image context

for file in files:
    k = file.stem
    d = data[k]
    print(f"Processing {k}")

    original_coords = d["original_coords"]
    relaxed_coords = d["relaxed_coords"]
    forces = d["relaxed_forces"]
    dualLengths = d["dualLengths"]
    f_mag = np.linalg.norm(forces, axis=1)

    norm = mpl.colors.Normalize(vmin=f_mag.min(), vmax=f_mag.max())
    # Map values to colors and add vertex color layer
    colors = cm(norm(f_mag))

    # with Image.open(f"crop_images/{k}.png") as im:
    with Image.open(f"raw_images/{k}.TIF") as im:
        pixel_scale = images[k]

        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
        # ax.plot(original_coords[:, 0], original_coords[:, 1], color="r")
        # ax.plot(relaxed_coords[:, 0], relaxed_coords[:, 1], color="gray", zorder=1)

        Q = ax.quiver(
            relaxed_coords[:, 0],
            relaxed_coords[:, 1],
            forces[:, 0],
            forces[:, 1],
            f_mag,
            cmap=cm,
            angles="xy",
            units="xy",
            label="force",
            scale=4e1,
            scale_units="xy",
            width=0.1,
            zorder=10,
        )
        ax.set_ylabel(r"X (μm)")
        ax.set_xlabel(r"Y (μm)")

        x_lim = np.array(ax.get_xlim()) + [-padding, padding]
        y_lim = np.array(ax.get_ylim()) + [-padding, padding]

        ax.set_xlim(x_lim)
        ax.set_ylim(y_lim)

        x_lim_pix = (x_lim / pixel_scale).round()
        y_lim_pix = (y_lim / pixel_scale).round()

        ax.set_ylim(ax.get_ylim()[::-1])

        im = im.crop((x_lim_pix[0], y_lim_pix[0], x_lim_pix[1], y_lim_pix[1]))

        plt.imshow(
            im,
            alpha=0.6,
            extent=(x_lim[0], x_lim[1], y_lim[1], y_lim[0]),
            zorder=0,
            cmap=plt.cm.Greys_r,
        )

        # Shrink current axis
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
        cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cm), ax=ax)
        cbar.ax.get_yaxis().labelpad = 20
        cbar.ax.set_ylabel("Force Density ($\mathregular{pN/\mu m^2}$)", rotation=270)

        # Quiver key draws length of arrow
        # qk = ax.quiverkey(Q, 0.45, 0.8, 500, r'$500 \frac{pN}{\mu m^2}$', labelpos='E', coordinates='figure')

        plt.savefig(f"figures/{file.stem}_force.pdf")
        plt.show()


In [ ]:
cm = mpl.cm.viridis_r

for file in files:
    k = file.stem
    d = data[k]
    print(f"Processing {k}")

    original_coords = d["original_coords"]
    relaxed_coords = d["relaxed_coords"]
    forces = d["relaxed_forces"]
    dualLengths = d["dualLengths"]
    f_mag = np.linalg.norm(forces, axis=1)

    norm = mpl.colors.Normalize(vmin=f_mag.min(), vmax=f_mag.max())
    # Map values to colors and add vertex color layer
    colors = cm(norm(f_mag))

    # with Image.open(f"crop_images/{k}.png") as im:
    with Image.open(f"raw_images/{k}.TIF") as im:
        pixel_scale = images[k]

        fig, ax = plt.subplots(1, 1, figsize=(5, 5))

        ax.plot(original_coords[:, 0], original_coords[:, 1], color="r")
        ax.plot(relaxed_coords[:, 0], relaxed_coords[:, 1], color="k", zorder=10)
        ax.set_ylabel(r"X (μm)")
        ax.set_xlabel(r"Y (μm)")

        x_lim = np.array(ax.get_xlim())
        y_lim = np.array(ax.get_ylim())

        x_lim_pix = (x_lim / pixel_scale).round()
        y_lim_pix = (y_lim / pixel_scale).round()

        ax.set_ylim(ax.get_ylim()[::-1])

        im = im.crop((x_lim_pix[0], y_lim_pix[0], x_lim_pix[1], y_lim_pix[1]))

        plt.imshow(
            im, alpha=0.6, extent=(x_lim[0], x_lim[1], y_lim[1], y_lim[0]), zorder=0, cmap = plt.cm.Greys_r
        )

        # Shrink current axis
        # box = ax.get_position()
        # ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
        # cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cm), ax=ax)
        # cbar.ax.get_yaxis().labelpad = 20
        # cbar.ax.set_ylabel("Force Density ($\mathregular{pN/\mu m^2}$)", rotation=270)

        plt.savefig(f"figures/{file.stem}_force.pdf")
        plt.show()


In [ ]:
for file in files:
    k = file.stem
    d = data[k]

    original_coords = d["original_coords"]
    relaxed_coords = d["relaxed_coords"]
    forces = d["relaxed_forces"]
    dualLengths = d["dualLengths"]

    print(
        "Mean dual length:",
        np.mean(dualLengths),
        "μm;",
        np.mean(dualLengths) ** 2,
        "μm^2",
    )


Assuming an actin filament is ~7nm in diameter a conservative surface area occupancy is 49 $\mathrm{nm^2}$ or 4.9e-5 $\mathrm{\mu m^2}$. Depending on cell conditions a single actin filament can produce ~1 pN of force.

Then 1pN * A / 4.9e-5 μm^2 gives the maximum force assuming full surface coverage by pushing actin.


